In [2]:
!pip install transformers timm 

  Using cached torch-2.8.0-cp312-cp312-win_amd64.whl.metadata (30 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ----------------------------- ---------- 1.8/2.5 MB 11.2 MB/s eta 0:00:01
   ---------------------------------------- 2.5/2.5 MB 9.7 MB/s  0:00:00
Using cached torch-2.8.0-cp312-cp312-win_amd64.whl (241.3 MB)
Using cached sympy-1.14.0-py3-none-any.whl (6.3 MB)

  Attempting uninstall: sympy

    Found existing installation: sympy 1.13.1

   ---------------------------------------- 0/3 [sympy]
   ---------------------------------------- 0/3 [sympy]
    Uninstalling sympy-1.13.1:
   ---------------------------------------- 0/3 [sympy]
   ---------------------------------------- 0/3 [sympy]
   ---------------------------------------- 0/3 [sympy]
   ---------------------------------------- 0/3 [sympy]
   ---------------------------------------- 0/3 [sympy]
   ------------------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unsloth-zoo 2025.10.1 requires transformers!=4.52.0,!=4.52.1,!=4.52.2,!=4.52.3,!=4.53.0,!=4.54.0,!=4.55.0,!=4.55.1,<=4.56.2,>=4.51.3, but you have transformers 4.57.0.dev0 which is incompatible.

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import models
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder # <-- Import ini

# --- 1. Konfigurasi dan Hiperparameter ---
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Menggunakan device: {DEVICE}")

IMAGE_SIZE = 224
# NUM_CLASSES akan kita dapatkan secara otomatis dari folder
# ... (Hiperparameter lainnya sama) ...
BATCH_SIZE = 32
EPOCHS_FEATURE_EXTRACTION = 3
EPOCHS_FINE_TUNING = 5
LR_FEATURE_EXTRACTION = 1e-3
LR_FINE_TUNING = 1e-5

# --- 2. Memuat Data Anda Menggunakan ImageFolder ---

# Tentukan path ke data Anda
DATA_DIR = './train_with_label' # <-- Ganti dengan path Anda

# Transformasi tetap sama (ViT butuh 224x224 dan normalisasi ImageNet)
transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Muat dataset menggunakan ImageFolder
# Asumsi: Anda belum punya folder validasi terpisah. Kita akan bagi nanti.
full_dataset = ImageFolder(root=DATA_DIR, transform=transform)
print(f"Dataset dimuat. Ditemukan {len(full_dataset)} gambar.")

# Dapatkan nama kelas dan jumlah kelas secara otomatis
class_names = full_dataset.classes
NUM_CLASSES = len(class_names)
print(f"Ditemukan {NUM_CLASSES} kelas: {class_names}")

# --- CATATAN PENTING: Anda perlu membagi data latih dan validasi ---
# Kode 'train_with_label' Anda menyiratkan itu semua data latih.
# Kita perlu membaginya, misal 80% latih, 20% validasi.

train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size])

print(f"Total data: {len(full_dataset)}")
print(f"Data latih: {len(train_dataset)}")
print(f"Data validasi: {len(val_dataset)}")

# Buat DataLoaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
# Kita ganti 'test_loader' menjadi 'val_loader' (loader validasi)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

# --- 3. Memuat Model Pre-trained ViT ---

# Muat model ViT-B/16 pre-trained
# (menggunakan API weights yang modern)
model = models.vit_b_16(weights=models.ViT_B_16_Weights.IMAGENET1K_V1)

# --- 4. Tahap 1: Feature Extraction (Melatih Head Saja) ---

print("\n--- Memulai Pelatihan Tahap 1 (Feature Extraction) ---")

# Bekukan semua parameter di base model
for param in model.parameters():
    param.requires_grad = False

# Ganti classifier head
# 'model.heads.head' adalah layer Linear(in_features=768, out_features=1000)
in_features = model.heads.head.in_features
model.heads.head = nn.Linear(in_features, NUM_CLASSES)

# Pindahkan model ke device
model = model.to(DEVICE)

# Tentukan loss function
criterion = nn.CrossEntropyLoss()

# Optimizer HANYA untuk parameter head yang baru (yang 'requires_grad=True')
# model.heads.head.parameters() adalah satu-satunya yang tidak dibekukan
optimizer = optim.Adam(model.heads.head.parameters(), lr=LR_FEATURE_EXTRACTION)

# Loop pelatihan sederhana
for epoch in range(EPOCHS_FEATURE_EXTRACTION):
    model.train() # Set model ke mode training
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)

        # Nol-kan gradien
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass dan optimasi
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if (i + 1) % 100 == 0: # Cetak setiap 100 batch
            print(f'[Epoch {epoch + 1}/{EPOCHS_FEATURE_EXTRACTION}, Batch {i + 1}] loss: {running_loss / 100:.3f}')
            running_loss = 0.0

print('Tahap 1 Selesai.')

# --- 5. Tahap 2: Fine-Tuning (Melatih Seluruh Model) ---

print("\n--- Memulai Pelatihan Tahap 2 (Fine-Tuning) ---")

# Buka (unfreeze) semua parameter
for param in model.parameters():
    param.requires_grad = True

# Buat optimizer baru untuk SEMUA parameter dengan learning rate SANGAT KECIL
optimizer = optim.Adam(model.parameters(), lr=LR_FINE_TUNING)

# Lanjutkan pelatihan
for epoch in range(EPOCHS_FINE_TUNING):
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if (i + 1) % 100 == 0:
            print(f'[Epoch {epoch + 1}/{EPOCHS_FINE_TUNING}, Batch {i + 1}] loss: {running_loss / 100:.3f}')
            running_loss = 0.0

print('Tahap 2 Selesai.')

# --- 6. Evaluasi Model ---
print("\n--- Evaluasi Model Final ---")
model.eval() # Set model ke mode evaluasi
correct = 0
total = 0

with torch.no_grad(): # Tidak perlu menghitung gradien saat evaluasi
    for data in test_loader:
        images, labels = data
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        
        outputs = model(images)
        
        # Ambil prediksi (kelas dengan probabilitas tertinggi)
        _, predicted = torch.max(outputs.data, 1)
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Akurasi pada 10000 gambar tes: {accuracy:.2f} %')

Menggunakan device: cpu
Dataset dimuat. Ditemukan 3650 gambar.
Ditemukan 15 kelas: ['Ayam Bakar', 'Ayam Betutu', 'Ayam Goreng', 'Ayam Pop', 'Bakso', 'Coto Makassar', 'Gado Gado', 'Gudeg', 'Nasi Goreng', 'Pempek', 'Rawon', 'Rendang', 'Sate Madura', 'Sate Padang', 'Soto']
Total data: 3650
Data latih: 2920
Data validasi: 730

--- Memulai Pelatihan Tahap 1 (Feature Extraction) ---
Tahap 1 Selesai.

--- Memulai Pelatihan Tahap 2 (Fine-Tuning) ---


KeyboardInterrupt: 